## In this example i will show how to fine-turn BERT model for multiclass text classification.

First we will import the library, we will use pytorch and keras, transformers for BERT and ML libraries.

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

#!pip install transformers==4.11.3
import transformers
print(transformers.__version__)

from transformers import BertTokenizer, BertForSequenceClassification

from transformers import AdamW

from tqdm import tqdm, trange
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import matthews_corrcoef

4.11.3


check if there is a gpu available

In [2]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2060


load the data, for this example we will use two datasets - one for train and validation and one for test the model

In [6]:
df_train = pd.read_csv('../bases/encuesta1.txt', sep='\t', encoding='latin1')  
df_train = df_train[['text','label']]


df_test=pd.read_csv('../bases/encuesta2.txt', sep='\t', encoding='latin1')  
df_test = df_test[['text','label']]

at first we will explore the adventages of BERT. This model is pre-trained and have tokenizers. We will check three differents models. Bert (original) Bert (Multilanguage) and Beto (a model trained with spanish vocabulary)

In [3]:
from transformers import BertTokenizer
tokenizerBERT = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizerMULTI = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
tokenizerBETO = BertTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis", do_lower_case=True)

In [4]:
#send the variables to a list to iterate

labels = df_train['label']
sentences = df_train['text']

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

print ("Original Text")
print (sentences[1])

tokenizedBERT_texts = [tokenizerBERT.tokenize(sent) for sent in sentences]
print ("Tokenize BERT BASE")
print (tokenizedBERT_texts[1])

tokenizerMULTI_texts = [tokenizerMULTI.tokenize(sent) for sent in sentences]
print ("Tokenize BERT MULTI")
print (tokenizerMULTI_texts[1])

tokenizerBETO_texts = [tokenizerBETO.tokenize(sent) for sent in sentences]
print ("Tokenize BETO")
print (tokenizerBETO_texts[1])

Original Text
[CLS] Es muy caro para lo que ofreces  [SEP]
Tokenize BERT BASE
['[CLS]', 'es', 'mu', '##y', 'car', '##o', 'para', 'lo', 'que', 'of', '##re', '##ces', '[SEP]']
Tokenize BERT MULTI
['[CLS]', 'es', 'muy', 'car', '##o', 'para', 'lo', 'que', 'ofrece', '##s', '[SEP]']
Tokenize BETO
['[CLS]', 'es', 'muy', 'caro', 'para', 'lo', 'que', 'ofrece', '##s', '[SEP]']


We check that the spanish tokenizer works better. So we decide to use this model.

In [23]:
tokenizer = BertTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis", do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

#Ademas cargo el modelo
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis", num_labels=3)

#model.to(device)
model.to('cpu')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31006, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

if we want to add more than one layer, we can create a class and use it as a model. In this ustomized model, i will add a drop out and a dense layer on top of bert to get the final output for the model. 

In [8]:

cat_response=3 #number of categories i want to predict
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained('finiteautomata/beto-sentiment-analysis', return_dict=True)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, cat_response)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
model = BERTClass()
device = 'cpu'
model.to(device)



Some weights of the model checkpoint at finiteautomata/beto-sentiment-analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31006, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

We already have our model created, we have to build the tensors to use in the model.

In [11]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 32

# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(input_ids[0])

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)


[4, 2981, 1992, 2645, 1042, 1456, 14850, 1110, 1030, 3285, 5]


In [24]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.1)


train_labels=train_labels.to_numpy()
validation_labels=validation_labels.to_numpy()


# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [25]:
# Defining some key variables that will be used later on in the training hyperparameters
MAX_LEN = 12
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 2
LEARNING_RATE = 1e-05



param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=1e-05,
                       correct_bias=False)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)





I can check the GPU Memory and free - In this case i will force to use the cpu

In [19]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

#def free_gpu_cache():
#    print("Initial GPU Usage")
#    gpu_usage()                            

#    torch.cuda.empty_cache()

#    cuda.select_device(0)
#    cuda.close()
#    cuda.select_device(0)

#    print("GPU Usage after emptying the cache")
#    gpu_usage()

#free_gpu_cache()                          
 
#model.cuda()


#model.cpu()
device='cpu'


finally, we train the model

In [26]:
#from transformers import TrainingArguments, Trainer
#from transformers import EarlyStoppingCallback


random.seed(1977)

train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
 
 
  # Training
 
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
 
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
 
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss[0].item())  
    # Backward pass
    loss[0].backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
   
    # Update tracking variables
    tr_loss += loss[0].item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
   
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.logits.detach().cpu().numpy()
    #logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))



print("Train loss: {}".format(tr_loss/nb_tr_steps))

   

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 2.606335163116455


Epoch:  25%|██▌       | 1/4 [00:19<00:58, 19.39s/it]

Validation Accuracy: 0.8
Train loss: 0.4461190899213155


Epoch:  50%|█████     | 2/4 [00:36<00:35, 17.83s/it]

Validation Accuracy: 0.9333333333333333
Train loss: 0.26629797286457485


Epoch:  75%|███████▌  | 3/4 [00:54<00:18, 18.27s/it]

Validation Accuracy: 1.0
Train loss: 0.09715864890151554


Epoch: 100%|██████████| 4/4 [01:15<00:00, 18.93s/it]

Validation Accuracy: 1.0
Train loss: 0.09715864890151554


Once the model is trained. We can test it again a new dataset to see the performance. 

In [27]:
###EN ESTA PARTE PROCESAMOS LOS DATOS NUEVOS PARA DARLE EL FORMATO NECESARIO PARA PYTORCH###

# Create sentence and label lists
sentences = df_test.text.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df_test.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 32
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 16  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)



###AQUI COMIENZA LA PREDICCIÓN###

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)



Lest check the performance of the model

In [28]:
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
    
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

print('matthews corrcoef: ',matthews_corrcoef(flat_true_labels, flat_predictions))



d = {'Labels':flat_true_labels,'Predictions':flat_predictions}
resumen = pd.DataFrame(d, columns=['Labels','Predictions'])


#print(pd.crosstab(resumen['Labels'], resumen['Predictions'] , dropna=False))

target_names = ['Servicio', 'Contenido', 'Precio']
print(classification_report(flat_true_labels, flat_predictions, target_names=target_names))


matthews corrcoef:  0.648044552662931
              precision    recall  f1-score   support

    Servicio       0.60      0.80      0.69        15
   Contenido       0.94      0.89      0.92        19
      Precio       0.75      0.56      0.64        16

    accuracy                           0.76        50
   macro avg       0.76      0.75      0.75        50
weighted avg       0.78      0.76      0.76        50



/home/juanpablosokil/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


we obtained a accuracy of 0.76. Lets explore the values:

In [30]:
df_test['predictions']=flat_predictions
df_test.head(10)
#we can save the predictions
#df_test.to_csv("testeo.csv")

,text,label,predictions
0,"No funciona el grabado, ni el reinicio y tampo...",0,0
1,Altos costos y poca actualizacion de peliculas,1,1
2,programacion basica y muy repetitiva. Sumado a...,2,1
3,no siempre funciona,0,0
4,no encuentro las peliculas que me interesan y ...,2,0
5,Cuando quiero ver alguna pelicula nueva tengo ...,2,2
6,No siempre se puede ver programas ya emitidos,0,0
7,Muchas veces no se puede ver un programa desde...,0,0
8,Bueno pero caro,1,1
9,"Viendo futbol de La Liga, River, se interrump...",0,0


In [31]:
# Saving the files for re-use

output_model_file = '../models/pytorch_textclasif.bin'
output_vocab_file = '../models/vocab_textclasif.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')


All files saved
This tutorial is completed


###  Once we have trained the model, we load it to apply to new data

In [4]:
output_model_file = '../models/pytorch_textclasif.bin'
output_vocab_file = '../models/vocab_textclasif.bin'

# Model class must be defined somewhere
model = torch.load(output_model_file)
tokenizer = BertTokenizer.from_pretrained(output_vocab_file)




/home/juanpablosokil/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1645: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [9]:
device='cpu'

# Create sentence and label lists
sentences = df_test.text.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df_test.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

MAX_LEN = 32
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 16  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)



###AQUI COMIENZA LA PREDICCIÓN###

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)



matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
    
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

print('matthews corrcoef: ',matthews_corrcoef(flat_true_labels, flat_predictions))



d = {'Labels':flat_true_labels,'Predictions':flat_predictions}
resumen = pd.DataFrame(d, columns=['Labels','Predictions'])


#print(pd.crosstab(resumen['Labels'], resumen['Predictions'] , dropna=False))

target_names = ['Servicio', 'Contenido', 'Precio']
print(classification_report(flat_true_labels, flat_predictions, target_names=target_names))


matthews corrcoef:  0.648044552662931
              precision    recall  f1-score   support

    Servicio       0.60      0.80      0.69        15
   Contenido       0.94      0.89      0.92        19
      Precio       0.75      0.56      0.64        16

    accuracy                           0.76        50
   macro avg       0.76      0.75      0.75        50
weighted avg       0.78      0.76      0.76        50



/home/juanpablosokil/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
